# Custom Layers

## Layers without Parameters

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now incorporate our layer as a component
in constructing more complex models.

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.2596e-09, grad_fn=<MeanBackward0>)

## Layers with Parameters

Now that we know how to define simple layers,
let us move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let us implement our own version of the  fully-connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires to input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [6]:
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.1300,  0.6243,  0.2260],
        [-0.2050,  0.8555,  1.4208],
        [-0.3892, -0.0839, -1.4606],
        [-1.9100,  1.2330,  1.6510],
        [ 1.7910, -0.9983, -1.2444]], requires_grad=True)

We can directly carry out forward propagation calculations using custom layers.

In [7]:
dense(torch.rand(2, 5))

tensor([[0.0000, 0.9926, 0.0000],
        [1.1913, 0.9176, 0.0000]])

We can also construct models using custom layers.

In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.8937],
        [8.1392]])

## Summary

* We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
* Once defined, custom layers can be invoked in arbitrary contexts and architectures.
* Layers can have local parameters, which can be created through built-in functions.

## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.